# TensorFlow Modelling
In this notebook, I did simple tensorflow model and I tried to embed but was not able to figure it out yet. 

In [2]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline

In [2]:
import tensorflow as tf
from tensorflow.sparse import SparseTensor
from tensorflow import keras
from tensorflow.keras.preprocessing.sequence import pad_sequences 
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Flatten, Dropout, Embedding
from tensorflow.keras.callbacks import ModelCheckpoint 
from sklearn.metrics import roc_auc_score, roc_curve 

Reading in the data.

In [3]:
X_train = pd.read_pickle('sparse_data.pkl')
y_train = pd.read_pickle('y_train.pkl')
X_test = pd.read_pickle('test_spasedata.pkl')
y_test = pd.read_pickle('y_test.pkl')

In [4]:
X_train.dtypes

vote_A     Sparse[int16, 0]
vote_N     Sparse[int16, 0]
vote_P     Sparse[int16, 0]
vote_Y     Sparse[int16, 0]
party_1    Sparse[int16, 0]
                 ...       
yukon      Sparse[int16, 0]
yy         Sparse[int16, 0]
zealand    Sparse[int16, 0]
érable     Sparse[int16, 0]
îles       Sparse[int16, 0]
Length: 5970, dtype: object

I want to get the data into a tensor but maintain it to be sparse.  I found a good piece of code to do so.

In [5]:
#Thanks stackoverflow : https://stackoverflow.com/questions/40896157/scipy-sparse-csr-matrix-to-tensorflow-sparsetensor-mini-batch-gradient-descent
def convert_sparse_matrix_to_sparse_tensor(coo):
    indices = np.mat([coo.row, coo.col]).transpose()
    return SparseTensor(indices, coo.data, coo.shape)

Converting it.

In [6]:
X_train_tensor = convert_sparse_matrix_to_sparse_tensor(X_train.sparse.to_coo())
X_test_tensor = convert_sparse_matrix_to_sparse_tensor(X_test.sparse.to_coo())

Running the model.

In [7]:
model = Sequential()
model.add(Dense(5970, activation='relu'))
model.add(Dropout(0.2))
model.add(Dense(2500, activation = 'relu'))
model.add(Dropout(0.1))
model.add(Dense(1, activation='sigmoid'))
model.compile(loss='binary_crossentropy', optimizer='adam',
              metrics=['accuracy'])

I got an error from tensorflow saying I should do this, this sets up the data into canonical order which tensorflow needs for sparse data.

In [8]:
X_train_tensor = tf.sparse.reorder(X_train_tensor)
X_test_tensor = tf.sparse.reorder(X_test_tensor)

In [9]:
model.fit(X_train_tensor, np.array(y_train),
              batch_size = 128,
              epochs = 3,
              verbose = 1,
              validation_data = (X_test_tensor, np.array(y_test)))

Epoch 1/3
3838/3838 [==============================] - 1431s 373ms/step - loss: 0.5629 - accuracy: 0.6823 - val_loss: 0.5770 - val_accuracy: 0.7971
Epoch 2/3
3838/3838 [==============================] - 1431s 373ms/step - loss: 0.5362 - accuracy: 0.6977 - val_loss: 0.6952 - val_accuracy: 0.7914
Epoch 3/3
3838/3838 [==============================] - 1404s 366ms/step - loss: 0.5326 - accuracy: 0.6996 - val_loss: 0.7217 - val_accuracy: 0.7489


Saving the model.

In [14]:
model.save('saved_model/my_model')

Instructions for updating:
If using Keras pass *_constraint arguments to layers.
INFO:tensorflow:Assets written to: saved_model/my_model\assets


In [11]:
y_test.mean()

0.8334894150577242

### Embedding

I attempt to embed here but was not able to finish this, I may try again in the future.

In [3]:
df_train = pd.read_pickle('text_train.pkl')
df_test = pd.read_pickle('text_test.pkl')

In [3]:
df_train

,politician_id,votequestion_id,vote,dissent,vote_question_id,vote_date,description_en,id,name,party_id,riding_id,election_id,elected,election_date,byelection,votepercent
0,77,540,N,True,540,2008-05-26,that the bill be now read a second time and r...,77,John Cummins,1,59006,1,True,2008-10-14,False,55.75
1,77,534,N,False,534,2008-05-07,that the bill be now read a second time and r...,77,John Cummins,1,59006,1,True,2008-10-14,False,55.75
2,77,496,Y,False,496,2008-03-12,"that, in the opinion of the house, the governm...",77,John Cummins,1,59006,1,True,2008-10-14,False,55.75
3,77,490,Y,False,490,2008-03-05,that the bill be now read a second time and r...,77,John Cummins,1,59006,1,True,2008-10-14,False,55.75
4,77,461,Y,True,461,2007-12-12,that the bill be now read a second time and r...,77,John Cummins,1,59006,1,True,2008-10-14,False,55.75
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
289087,9294,2366,N,False,2366,2015-06-17,"that, in the opinion of the house: ( a) the ...",9294,Jim Eglinski,1,48028,32,True,2015-10-19,False,72.30
289088,9294,2367,Y,False,2367,2015-06-17,that the 21st report of the standing committee...,9294,Jim Eglinski,1,48028,32,True,2015-10-19,False,72.30
289089,9294,2368,N,False,2368,2015-06-18,that the motion be amended by deleting all the...,9294,Jim Eglinski,1,48028,32,True,2015-10-19,False,72.30
289090,9294,2369,Y,False,2369,2015-06-18,that the bill be now read a third time and do ...,9294,Jim Eglinski,1,48028,32,True,2015-10-19,False,72.30


In [4]:
X_train = df_train['description_en']
y_train = df_train['elected'].astype('int')
X_test = df_test['description_en']
y_test = df_test['elected'].astype('int')

In [5]:
X_train = X_train.str.replace('\d+', '')
X_test = X_test.str.replace('\d+', '')

In [6]:
X_train = X_train.str.replace('$', '')
X_test = X_test.str.replace('$', '')

In [7]:
X_train = X_train.str.replace('  ', ' ')
X_test = X_test.str.replace('  ', ' ')

In [8]:
from tensorflow.python.keras.preprocessing.text import Tokenizer
from tensorflow.python.keras.preprocessing.sequence import pad_sequences
#define vocabulary size
vocab_size = 4000
tokenizer_obj = Tokenizer(num_words=vocab_size)
total_reviews = X_train.append(X_test)

In [9]:
tokenizer_obj.fit_on_texts(list(total_reviews))

In [10]:
# pad sequences
max_length = max([len(s.split()) for s in total_reviews])

X_train_tokens = tokenizer_obj.texts_to_sequences(X_train)
X_test_tokens = tokenizer_obj.texts_to_sequences(X_test)

X_train_pad = pad_sequences(X_train_tokens, maxlen=max_length, padding='post')
X_test_pad = pad_sequences(X_test_tokens, maxlen=max_length, padding='post')

In [8]:
from tensorflow.keras import Model, Sequential
from tensorflow.keras.layers import Activation, Dense, Dot, Embedding, Flatten, GlobalAveragePooling1D, Reshape, GRU

model = Sequential()
model.add(Embedding(vocab_size, 100, input_length=max_length))
model.add(GRU(units=32, dropout=0.2, recurrent_dropout=0.2))
model.add(Dense(1, activation='sigmoid'))

model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])

In [ ]:
model.fit(X_train_pad, y_train, batch_size=128, epochs=8, validation_data=(X_test_pad, y_test), verbose=1)

Epoch 1/8
3838/3838 [==============================] - 10069s 3s/step - loss: 0.6688 - accuracy: 0.6106 - val_loss: 0.5550 - val_accuracy: 0.8335
Epoch 2/8
3838/3838 [==============================] - 9554s 2s/step - loss: 0.6686 - accuracy: 0.6106 - val_loss: 0.5901 - val_accuracy: 0.8335
Epoch 3/8
 373/3838 [=>............................] - ETA: 2:18:59 - loss: 0.6691 - accuracy: 0.6096

The model does not appear to be learning, I shall stop it here and try another one.

In [13]:
from tensorflow.keras import Model, Sequential
from tensorflow.keras.layers import Activation, Dense, Dot, Embedding, Flatten, GlobalAveragePooling1D, Reshape, GRU
from tensorflow.keras import layers

model2 = Sequential()
model2.add(Embedding(vocab_size, 100, input_length=max_length))
model2.add(layers.Bidirectional(layers.LSTM(32)))
model2.add(Dense(1, activation='sigmoid'))

model2.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])

In [ ]:
model2.fit(X_train_pad, y_train, batch_size=512, epochs=3, validation_data=(X_test_pad, y_test), verbose=1)

Epoch 1/3
702/960 [====================>.........] - ETA: 3:56:55 - loss: 0.6461 - accuracy: 0.6104

This crashed, I don't may try it again later, but right now I wish to look at something more efficient.

In [11]:
from tensorflow.keras import Model, Sequential
from tensorflow.keras.layers import Activation, Dense, Dot, Embedding, Flatten, GlobalAveragePooling1D, Reshape, GRU
from tensorflow.keras import layers

model3 = Sequential()
model3.add(Embedding(vocab_size, 100, input_length=max_length))
model3.add(layers.LSTM(32))
model3.add(Dense(1, activation='sigmoid'))

model3.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])

In [ ]:
model3.fit(X_train_pad, y_train, batch_size=512, epochs=3, validation_data=(X_test_pad, y_test), verbose=1)

Epoch 1/3
960/960 [==============================] - 8393s 9s/step - loss: 0.6688 - accuracy: 0.6104 - val_loss: 0.5783 - val_accuracy: 0.8335
Epoch 2/3
 32/960 [>.............................] - ETA: 2:02:44 - loss: 0.6684 - accuracy: 0.6111

In [8]:
X_train2 = df_train[df_train['vote'] == 'Y']['description_en']
y_train2 = df_train[df_train['vote'] == 'Y']['elected'].astype('int')
X_test2 = df_test[df_test['vote'] == 'Y']['description_en']
y_test2 = df_test[df_test['vote'] == 'Y']['elected'].astype('int')

In [9]:
from tensorflow.python.keras.preprocessing.text import Tokenizer
from tensorflow.python.keras.preprocessing.sequence import pad_sequences
#define vocabulary size
vocab_size = 4000
tokenizer_obj = Tokenizer(num_words=vocab_size)
total_reviews = X_train2.append(X_test2)

In [10]:
tokenizer_obj.fit_on_texts(list(total_reviews))

In [12]:
# pad sequences
max_length = max([len(s.split()) for s in total_reviews])

X_train_tokens = tokenizer_obj.texts_to_sequences(X_train2)
X_test_tokens = tokenizer_obj.texts_to_sequences(X_test2)

X_train_pad = pad_sequences(X_train_tokens, maxlen=max_length, padding='post')
X_test_pad = pad_sequences(X_test_tokens, maxlen=max_length, padding='post')

In [13]:
from tensorflow.keras import Model, Sequential
from tensorflow.keras.layers import Activation, Dense, Dot, Embedding, Flatten, GlobalAveragePooling1D, Reshape, GRU

model = Sequential()
model.add(Embedding(vocab_size, 100, input_length=max_length))
model.add(GRU(units=32, dropout=0.2, recurrent_dropout=0.2))
model.add(Dense(1, activation='sigmoid'))

model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])

In [14]:
model.fit(X_train_pad, y_train2, batch_size=128, epochs=3, validation_data=(X_test_pad, y_test), verbose=1)

Epoch 1/3
1871/1871 [==============================] - 3933s 2s/step - loss: 0.6727 - accuracy: 0.6016 - val_loss: 0.5841 - val_accuracy: 0.8121
Epoch 2/3
1871/1871 [==============================] - 3927s 2s/step - loss: 0.6726 - accuracy: 0.6016 - val_loss: 0.5851 - val_accuracy: 0.8121
Epoch 3/3
1871/1871 [==============================] - 3938s 2s/step - loss: 0.6725 - accuracy: 0.6016 - val_loss: 0.5995 - val_accuracy: 0.8121


In [15]:
y_train2.mean()

0.601588567658199

Again it looks like the model wasn't able to find any differences